## Import modules

In [ ]:
!pip install wordcloud
!pip install nltk
import nltk
nltk.download('stopwords')

import pandas as pd 
import numpy as np 
import time

#For Text Cleaning
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

#For EDA
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

#For text vectorizing
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora
from gensim import models
import random

#For naives bayes DF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix   

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


# Emotion Analysis 
# import text2emotion as te

# pos tagging 
from collections import Counter

# ploting 
import seaborn as sn
import matplotlib.pyplot as plt

# VADER generation 
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
import sys
import json
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

## Load Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CompiledNewsDataIndexed.csv")
df.head(3)

,title,text,label,index
0,Senate bill would count munis toward bank liqu...,"Bonds sold by U.S. states, cities, schools and...",REAL,8003
1,Germany to reach out to Trump government to ke...,Germany wants to reach out to the future U.S. ...,REAL,6254
2,"Turkey to review 11,480 cases linked to app us...",Turkish prosecutors said on Wednesday they wil...,REAL,11307


## Fake News Entity Recognition

In [ ]:
# Get fake news article dataframe
df1fake = df[df['label'] == 'FAKE']

# Get index of fake dataframe
index_list = df1fake.index.tolist()

In [ ]:
fakeEntityDic = getEntityDic(df1fake)
with open('fake_entity_dic.json', 'w') as fp:
    json.dump(fakeEntityDic, fp)

In [ ]:
trt = getTopEntityDic(fakeEntityDic, 10)

PERSON
----------------
trump                                    :17691
clinton                                  :5209
donald trump                             :4867
obama                                    :4179
hillary clinton                          :2897
hillary                                  :2476
donald                                   :1340
fox news                                 :779
barack obama                             :756
twitter                                  :726

ORGANIZATION
----------------
republican                               :2829
fbi                                      :2651
trump                                    :1939
gop                                      :1740
congress                                 :1449
cnn                                      :1370
video                                    :1296
democratic                               :1212
senate                                   :1186
us                                       :1108

GPE
--

In [ ]:
df1_fakeEntities = addEntitiesToDataframe2(df1fake.copy(), trt)
df1_fakeEntities.to_csv("df1_fakeEntities_30.csv", index = False)

Completion progress: 9821 of 9821 articles

In [ ]:
finalEntitydf

,title,text,label,index,person entities,organisation entities,location entities,person entity count,organisation entity count,location entity count,(GPE)germany,(GPE)south,(GPE)turkey,(GPE)ankara,(GPE)senate,(GPE)muni,(GPE)new york democrat,(GPE)schumer,(GPE)washington,(GPE)merkel,(ORGANIZATION)germany,(ORGANIZATION)south,(ORGANIZATION)bylock,(ORGANIZATION)turkey,(ORGANIZATION)ankara,(ORGANIZATION)senate,(ORGANIZATION)nast,(ORGANIZATION)greenpeace africa,(ORGANIZATION)muni,(ORGANIZATION)new york democrat,(PERSON)germany,(PERSON)south,(PERSON)bylock,(PERSON)turkey,(PERSON)ankara,(PERSON)senate,(PERSON)nast,(PERSON)greenpeace africa,(PERSON)africa,(PERSON)muni,(PERSON)trump,(PERSON)gingrich,(PERSON)clinton,(PERSON)kelly,(PERSON)schultz,(PERSON)donald trump,(PERSON)jeh johnson,(PERSON)corey lewandowski,(PERSON)bill clinton,(PERSON)johnson,(ORGANIZATION)trump,(ORGANIZATION)gingrich,(ORGANIZATION)clinton,(ORGANIZATION)kelly,(ORGANIZATION)dnc,(ORGANIZATION)republican,(ORGANIZATION)schultz,(ORGANIZATION)donald trump,(ORGANIZATION)jeh johnson,(ORGANIZATION)corey lewandowski,(GPE)gingrich,(GPE)clinton,(GPE)dnc,(GPE)republican,(GPE)schultz,(GPE)donald trump,(GPE)jeh johnson,(GPE)corey lewandowski,(GPE)bill clinton,(GPE)dhs
0,Senate bill would count munis toward bank liqu...,"Bonds sold by U.S. states, cities, schools and...",REAL,8003,"{'chuck schumer': 1, 'mike rounds': 1, 'dakota...","{'hqlas': 1, 'senators mark warner': 1, 'virgi...","{'senate': 1, 'muni': 1, 'new york democrat': ...",4.0,10.0,6.0,0,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Germany to reach out to Trump government to ke...,Germany wants to reach out to the future U.S. ...,REAL,6254,"{'donald trump': 1, 'chancellor angela merkel'...",{'nato': 1},"{'germany': 5, 'merkel': 1, 'united states': 1}",5.0,1.0,7.0,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Turkey to review 11,480 cases linked to app us...",Turkish prosecutors said on Wednesday they wil...,REAL,11307,{'yuksel kocaman': 1},"{'bylock': 4, 'fethullah gulen': 1}","{'turkish': 1, 'turkey': 3, 'ankara': 2, 'gule...",1.0,5.0,7.0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5 things to watch in tonight’s GOP debate,"Killing Obama administration rules, dismantlin...",REAL,2688,"{'obama': 1, 'obamacare': 1}",{'gop': 1},{},2.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Activists to appeal new South African nuclear ...,- Greenpeace Africa and other NGOs intend to a...,REAL,14941,"{'cape town': 1, 'eskom': 1, 'koeberg': 1, 'af...","{'greenpeace africa': 2, 'ngos': 1, 'edf': 1, ...","{'south africa': 1, 'south': 3, 'pretoria': 1,...",8.0,5.0,10.0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,DEBBIE WASSERMAN SCHULTZ Accuses Obama’s DHS D...,Rep. Debbie Wasserman Schultz dropped a big bo...,FAKE,10557,"{'debbie': 1, 'wasserman schultz': 1, 'jeh joh...","{'cnn': 2, 'dnc': 5, 'fbi': 2, 'dhs': 3, 'wiki...","{'whoa': 1, 'russians': 1}",14.0,15.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Trump Gets P*SSED As TODAY Hosts Slam His ‘Ly...,A smart presidential candidate would immediate...,FAKE,7206,"{'donald trump': 1, 'corey lewandowski': 3, 't...","{'hosts.host savannah guthrie': 1, 'republican...","{'savannah': 1, 'united states': 1, 'american'...",17.0,5.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,get ready for a likely market crash after elec...,by gordon duff senior editor on october radi...,FAKE,1830,{},{},{},0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
finalEntitydf = finalEntitydf[asd]
finalEntitydf.head()

,title,text,label,index,person entities,organisation entities,location entities,person entity count,organisation entity count,location entity count,"(True,G) united states","(True,G) u.s","(True,G) russia","(True,G) china","(True,G) iran","(True,G) trump","(True,G) washington","(True,G) russian","(True,G) north korea","(True,G) american","(True,O) republican","(True,O) united states","(True,O) u.s","(True,O) senate","(True,O) house","(True,O) russia","(True,O) china","(True,O) congress","(True,O) democratic","(True,O) iran","(True,P) trump","(True,P) clinton","(True,P) republican","(True,P) united states","(True,P) u.s","(True,P) donald trump","(True,P) senate","(True,P) house","(True,P) russia","(True,P) china","(Fake,P) trump","(Fake,P) clinton","(Fake,P) donald trump","(Fake,P) obama","(Fake,P) hillary clinton","(Fake,P) hillary","(Fake,P) donald","(Fake,P) fox news","(Fake,P) barack obama","(Fake,P) twitter","(Fake,O) clinton","(Fake,O) donald trump","(Fake,O) obama","(Fake,O) hillary clinton","(Fake,O) republican","(Fake,O) fbi","(Fake,O) hillary","(Fake,O) trump","(Fake,O) gop","(Fake,O) congress","(Fake,G) clinton","(Fake,G) donald trump","(Fake,G) obama","(Fake,G) trump","(Fake,G) america","(Fake,G) american","(Fake,G) hillary clinton","(Fake,G) republican","(Fake,G) russia","(Fake,G) fbi"
0,Senate bill would count munis toward bank liqu...,"Bonds sold by U.S. states, cities, schools and...",REAL,8003,"{'chuck schumer': 1, 'mike rounds': 1, 'dakota...","{'hqlas': 1, 'senators mark warner': 1, 'virgi...","{'senate': 1, 'muni': 1, 'new york democrat': ...",4.0,10.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Germany to reach out to Trump government to ke...,Germany wants to reach out to the future U.S. ...,REAL,6254,"{'donald trump': 1, 'chancellor angela merkel'...",{'nato': 1},"{'germany': 5, 'merkel': 1, 'united states': 1}",5.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Turkey to review 11,480 cases linked to app us...",Turkish prosecutors said on Wednesday they wil...,REAL,11307,{'yuksel kocaman': 1},"{'bylock': 4, 'fethullah gulen': 1}","{'turkish': 1, 'turkey': 3, 'ankara': 2, 'gule...",1.0,5.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5 things to watch in tonight’s GOP debate,"Killing Obama administration rules, dismantlin...",REAL,2688,"{'obama': 1, 'obamacare': 1}",{'gop': 1},{},2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Activists to appeal new South African nuclear ...,- Greenpeace Africa and other NGOs intend to a...,REAL,14941,"{'cape town': 1, 'eskom': 1, 'koeberg': 1, 'af...","{'greenpeace africa': 2, 'ngos': 1, 'edf': 1, ...","{'south africa': 1, 'south': 3, 'pretoria': 1,...",8.0,5.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Helper Functions

In [ ]:
def getFinalEntityDF(finalEntitydf, store):
  total_records = finalEntitydf.shape[0]
  for index, row in finalEntitydf.iterrows():
    sys.stdout.write('\rCompletion progress: ' + str(index+1) + ' of ' + str(total_records) + " articles")
    sys.stdout.flush()
    # Person Entities
    json_string = row['person entities']
    dicp = ast.literal_eval(json_string)
    for x in dicp:
      item = "(PERSON)" + str(x)
      if item in store:
        finalEntitydf.loc[index, item] = dicp[x]

    # Location Entities
    json_string = row['location entities']
    dicp = ast.literal_eval(json_string)
    for x in dicp:
      item = "(GPE)" + str(x)
      if item in store:
        finalEntitydf.loc[index, item] = dicp[x]
    
    # GPE Entities
    json_string = row['organisation entities']
    dicp = ast.literal_eval(json_string)
    for x in dicp:
      item = "(ORGANISATION)" + str(x)
      if item in store:
        finalEntitydf.loc[index, item] = dicp[x]

  return finalEntitydf

In [ ]:
def attachTopEntities(df, fakeTop, realTop):
  store = []
  for x in trueTop:
    for y in trueTop[x]:
      store.append(("(" + str(x) + ")" + y))

  for x in fakeTop:
    for y in fakeTop[x]:
      item = ("(" + str(x) + ")" + y)
      if item not in store:
        store.append(item)

  for x in store:
    df[x] = 0

  return df, store

In [ ]:
def addEntitiesToDataframe2(df):

    entity_dic = {}
    counter = 1
    total_records = df.shape[0]
    i = 0
    for index, row in df.iterrows():
        entity_dic['PERSON'] = {}
        entity_dic['ORGANIZATION'] = {}
        entity_dic['GPE'] = {}
        text = row['text'] + " "+ row['title']
        sys.stdout.write('\rCompletion progress: ' + str(counter) + ' of ' + str(total_records) + " articles")
        sys.stdout.flush()

        ## Put full stop behind each sentence
        new = []
        if "\n" in text:
          text = text.replace(" \n", ". ")
        text_list = text.split(". ")
        for x in text_list:
            new.append(x + ".")

        ## Cycle through each sentence
        for x in new:
            sent_tokens = word_tokenize(x)
            tagged_sent = nltk.pos_tag(sent_tokens)
            ne_tree = nltk.ne_chunk(tagged_sent)
            ne_list = extract_ne_from_tree(ne_tree)

            ## Insert into dictionary
            for y in ne_list:
                if y[0] not in entity_dic:
                    entity_dic[y[0]] = {}
                if y[0] in entity_dic:
                    string = y[1].lower()
                    if string not in entity_dic[y[0]]:
                        entity_dic[y[0]][string] = 0
                    if string in entity_dic[y[0]]:
                        entity_dic[y[0]][string] += 1
        counter += 1

        # Get df index
        ind = index_list[i]

        # get summation values
        total_p = 0
        for x in entity_dic['PERSON']:
          total_p += entity_dic['PERSON'][x]

        total_o = 0
        for x in entity_dic['ORGANIZATION']:
          total_o += entity_dic['ORGANIZATION'][x]

        total_g = 0
        for x in entity_dic['GPE']:
          total_g += entity_dic['GPE'][x]
        
        # insert into DF
        df.loc[ind:ind+1, 'person entities'] = str(entity_dic['PERSON'])
        df.loc[ind:ind+1, 'organisation entities'] = str(entity_dic['ORGANIZATION'])
        df.loc[ind:ind+1, 'location entities'] = str(entity_dic['GPE'])
        df.loc[ind:ind+1, 'person entity count'] = total_p
        df.loc[ind:ind+1, 'organisation entity count'] = total_o
        df.loc[ind:ind+1, 'location entity count'] = total_g
        entity_dic = {}
        i+=1
        
    return df

In [ ]:
def addEntitiesToDataframe(df, top10):

    # # create top entities columns in dataframe
    for en in top10:
      df[en] = 0

    entity_dic = {}
    counter = 1
    total_records = df.shape[0]
    i = 0
    for index, row in df.iterrows():
        entity_dic['PERSON'] = {}
        entity_dic['ORGANIZATION'] = {}
        entity_dic['GPE'] = {}
        text = row['text'] + " "+ row['title']
        sys.stdout.write('\rCompletion progress: ' + str(counter) + ' of ' + str(total_records) + " articles")
        sys.stdout.flush()

        ## Put full stop behind each sentence
        new = []
        if "\n" in text:
          text = text.replace(" \n", ". ")
        text_list = text.split(". ")
        for x in text_list:
            new.append(x + ".")

        ## Cycle through each sentence
        for x in new:
            sent_tokens = word_tokenize(x)
            tagged_sent = nltk.pos_tag(sent_tokens)
            ne_tree = nltk.ne_chunk(tagged_sent)
            ne_list = extract_ne_from_tree(ne_tree)

            ## Insert into dictionary
            for y in ne_list:
                if y[0] not in entity_dic:
                    entity_dic[y[0]] = {}
                if y[0] in entity_dic:
                    string = y[1].lower()
                    if string not in entity_dic[y[0]]:
                        entity_dic[y[0]][string] = 0
                    if string in entity_dic[y[0]]:
                        entity_dic[y[0]][string] += 1
        counter += 1
        # Get df index
        ind = index_list[i]

        # get summation values
        total_p = 0
        for x in entity_dic['PERSON']:
          total_p += entity_dic['PERSON'][x]
          if x in top10:
            df.loc[ind:ind+1, str(x)] = int(entity_dic['PERSON'][x])

        total_o = 0
        for x in entity_dic['ORGANIZATION']:
          total_o += entity_dic['ORGANIZATION'][x]
          if x in top10:
            df.loc[ind:ind+1, x] = entity_dic['ORGANIZATION'][x]

        total_g = 0
        for x in entity_dic['GPE']:
          total_g += entity_dic['GPE'][x]
          if x in top10:
            df.loc[ind:ind+1, x] = entity_dic['GPE'][x]
        
        # insert into DF
        df.loc[ind:ind+1, 'person entities'] = str(entity_dic['PERSON'])
        df.loc[ind:ind+1, 'organisation entities'] = str(entity_dic['ORGANIZATION'])
        df.loc[ind:ind+1, 'location entities'] = str(entity_dic['GPE'])
        df.loc[ind:ind+1, 'person entity count'] = total_p
        df.loc[ind:ind+1, 'organisation entity count'] = total_o
        df.loc[ind:ind+1, 'location entity count'] = total_g
        entity_dic = {}
        i+=1
        
    return df

In [ ]:
def extract_ne_from_tree ( tree ):
    result = []
    for s in tree.subtrees():
        label = s.label()
        if (label == 'PERSON' or label == 'ORGANIZATION' or label == 'GPE'):
            leaves = s.leaves()
            ne = ''
            for l in leaves:
                ne = ne + ' ' + l[0]
            result.append((label, ne[1:]))
    return result

In [ ]:
def getEntityDic(df):
  entity_dic = {}
  counter = 1
  total_records = df.shape[0]
  for index, row in df.iterrows():
    text = row['text'] + " "+ row['title']
    sys.stdout.write('\rCompletion progress: ' + str(counter) + ' of ' + str(total_records) + " articles")
    sys.stdout.flush()

    ## Put full stop behind each sentence
    new = []
    if "\n" in text:
        text = text.replace(" \n", ". ")
    text_list = text.split(". ")
    for x in text_list:
        new.append(x + ".")
    
    ## Cycle through each sentence
    for x in new:
        sent_tokens = word_tokenize(x)
        tagged_sent = nltk.pos_tag(sent_tokens)
        ne_tree = nltk.ne_chunk(tagged_sent)
        ne_list = extract_ne_from_tree(ne_tree)
        
        ## Insert into dictionary
        for y in ne_list:
            if y[0] not in entity_dic:
                entity_dic[y[0]] = {}
            if y[0] in entity_dic:
                string = y[1].lower()
                if string not in entity_dic[y[0]]:
                    entity_dic[y[0]][string] = 0
                if string in entity_dic[y[0]]:
                    entity_dic[y[0]][string] += 1
    counter += 1
  return entity_dic

In [ ]:
def getTopEntities(entity_dic, n):
  new_n = 30
  top10entities = {}
  for x in entity_dic:
    print(x)
    print("----------------")
    sorted_dic = sorted(entity_dic[x], key=entity_dic[x].get, reverse=True)
    count = 0
    for y in sorted_dic:
      top10entities[y] = entity_dic[x][y]
      if count <= new_n:
        print(y.ljust(40), ":" + str(entity_dic[x][y]))
      count += 1
      if count == n:
          break
    print()

  top10 = dict(sorted(top10entities.items(), key=lambda item: item[1], reverse = True))
  count = 0
  result = []
  for x in top10:
    count += 1
    result.append(x)
    if count == 50:
      break

  return result

In [ ]:
def getTopEntityDic(entity_dic, n):
  bigstore = {}
  top10entities = {}
  for x in entity_dic:
    first = str(x)[0] + "_"
    bigstore[x] = []
    print(x)
    print("----------------")
    sorted_dic = sorted(entity_dic[x], key=entity_dic[x].get, reverse=True)
    count = 0
    for y in sorted_dic:
      top10entities[y] = entity_dic[x][y]
      print(y.ljust(40), ":" + str(entity_dic[x][y]))
      count += 1
      if count == n:
          break
    top10 = dict(sorted(top10entities.items(), key=lambda item: item[1], reverse = True))
    count = 0
    result = []
    for z in top10:
      count += 1
      result.append(str(z))
      if count == n:
        bigstore[x] = result 
        break
    print()

  

  return bigstore